### Data manipulation: Sales data of small manufacturing company 

For this side project, I have been tasked with creating a dashboard for a small candle manufacturing company. The dashboard should have the following capabilities:
- Easily tracking monthly performance with the ability to update data with ease.
- Display of yearly performance for quick and easy analysis.
- Detailed visualization of sales at both individual product and customer levels.

I have opted to use Power BI for creating the dashboard due to its superior flexibility in accommodating diverse data schemas and advanced time intelligence functions.

In [10]:
import pandas as pd
data = pd.read_csv('messydata.csv', header=None)
data = data.reset_index()
data = data.drop(data.columns[0:43], axis=1)
data

,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,�Թ���,A153,NO.7 ��¹���Ό�� ᴧ,12/01/2562,DS256201/00119,C32,��������§���,���,20,-,32,-,640,-,640,-,640,HO
1,14/06/2562,DS256206/00057,C2,������,���,40,-,32,-,1280,-,1280,-,1280,HO,��� ������,7200131.5,0
2,13/11/2562,DS256211/00041,C32,��������§���,���,10,-,32,-,320,-,320,-,320,HO,��� ������,7200131.5,0
3,14/11/2562,DS256211/00043,C2,������,�ѧ40,1,-,1280,-,1280,-,1280,-,1280,HO,��� ������,7200131.5,0
4,13/01/2563,DS256301/00166,C21,����駡��,�ѧ40,1,-,1280,-,1280,-,1280,-,1280,HO,��� ������,7200131.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,22/12/2564,DS256412/00056,C22,�������,�ѧ,1,-,925,-,925,-,925,-,925,HO,��� ������,7200131.5,0
562,24/12/2564,DS256412/00085,C7,����,�ѧ,3,-,925,-,2775,222,2553,-,2553,HO,��� ������,7200131.5,0
563,20/01/2565,DS256501/00126,C35,���������,�ѧ,1,-,925,-,925,-,925,-,925,HO,��� ������,7200131.5,0
564,12/02/2565,DS256502/00066,C68,�ҫ���Ҹ�,�ѧ,1,-,925,-,925,-,925,-,925,HO,��� ������,7200131.5,0


The text in the dataset is in the different language, which means that we will need to change the encoding so that Power BI can read it. **(i will not fix this issue since it would reveal sensitive information.)**

The sales dataset that I have obtained from the candle manufacturing company's ERP system has several issues that need to be resolved before we can begin creating the dashboard. These issues include:

- The data is not organized into proper columns, with each entry lacking an assigned product ID. Instead, product IDs were given at the beginning of the entries. This disorganization makes it difficult to analyze the data properly, and we will need to restructure the data before we can proceed.

- The dates in the dataset are recorded in Buddhist year format, which may not be recognized by Power BI. Therefore, we will need to convert the date format into a format that can be recognized by Power BI.

- It appears that the individual product IDs can be sold either in bulk/crate or individual units, which could cause problems when calculating the quantity sold. We will need to address this issue to ensure that our sales analysis is accurate and reflects the actual sales trends.


**Remark:** To maintain the confidentiality of the company's data, I have decided not to provide the full dataset and we will only need its ids for the refined dataset. 


**Issue 1:** Data is not organized into proper columns

To do this, I created a new dataframe and used a for loop function to iterate over the column that contained the mixed product ID and invoice ID information. If the loop found a product ID, it saved the ID as a variable and assigned it to all the following rows until a new product ID was found to replace it.

After assigning the product IDs, the next step was to relocate the cell values into their appropriate columns. To achieve this, I created a new dataframe and rearranged the data by placing all the values in their respective columns. This process helped us structure the data in a more organized and useful manner that could be easily analyzed and used for creating the dashboard.

In [14]:
cleandata = pd.DataFrame(columns=['invoice_date', 'invoiceid', 'customer_id','product_id', 'quantity','unit_name', 'unit_price', 'discount', 'revenue']) 

for i in range(0,data.shape[0]):
            
            row = str(data.iloc[i,1])
            
            if row.startswith(('A', 'B', 'C', 'T')): #if row has initial code product category 
                #inserting filtered data into new data
                productid = data.iloc[i,1]
                cleandata.loc[i,'invoice_date'] = data.iloc[i,3]
                cleandata.loc[i,'invoiceid'] = data.iloc[i,4]
                cleandata.loc[i,'customer_id'] = data.iloc[i,5]
                cleandata.loc[i,'product_id'] = productid
                cleandata.loc[i,'quantity'] = data.iloc[i,8]
                cleandata.loc[i,'unit_name'] = data.iloc[i,7]
                cleandata.loc[i,'unit_price'] = data.iloc[i,10]
                cleandata.loc[i,'discount'] = data.iloc[i,13]
                cleandata.loc[i,'revenue'] = data.iloc[i,14]
                
            
            elif row.startswith(("DS","VAT")): #if row has invoiceID
                cleandata.loc[i,'invoice_date'] = data.iloc[i,0]
                cleandata.loc[i,'invoiceid'] = data.iloc[i,1]
                cleandata.loc[i,'customer_id'] = data.iloc[i,2]
                cleandata.loc[i,'product_id'] = productid
                cleandata.loc[i,'quantity'] = data.iloc[i,5]
                cleandata.loc[i,'unit_name'] = data.iloc[i,4]
                cleandata.loc[i,'unit_price'] = data.iloc[i,7]
                cleandata.loc[i,'discount'] = data.iloc[i,10]
                cleandata.loc[i,'revenue'] = data.iloc[i,11]

**Issue 2:** Buddhist year

The code used to convert the Buddhist year to the Christian year was relatively straightforward. It simply subtracted 543 from the Buddhist year to obtain the corresponding Christian year.


In [7]:
#Get A Distinct Year Value from InvoiceDate
distinctyear = cleandata['invoice_date'].str.slice(-4)
distinctyear = distinctyear.drop_duplicates()
distinctyear = distinctyear.tolist()

#Loop for Conversion
thai = sorted(distinctyear)
chrs = []

for i in thai:
    c = int(i) - 543
    chrs.append(str(c))
year = dict(zip(thai,chrs))


#Replacing Strings
cleandata['invoice_date'] = cleandata['invoice_date'].replace(year,regex=True)

#Convert String Invoice_Date to Datetime64[ns]
cleandata['invoice_date'] = pd.to_datetime(cleandata['invoice_date'], format = '%d/%m/%Y', dayfirst=True)

To address the problem of units sold in bulk orders versus individual units, I have decided to break down all bulk orders into their sub-units. This approach will make it easier to accurately calculate the total quantity sold.

Fortunately, the company's unit system also includes a subunit number at the end of the bulk unit, which means that we can easily extract this number and use it to calculate the number of individual units sold. This method will ensure that our sales analysis is accurate and reflects the actual sales trends for each product.

In [11]:
#Solving the issue with the unit discrepancies

# unitlists = cleandata[['product_id','unit_name']]
# unitlists = unitlists.drop_duplicates(['product_id','unit_name'])
# unitlists.to_excel('unitlists.xlsx')

# units = pd.read_excel('unitlists.xlsx')
# cleandata2 = pd.merge(cleandata, units, left_on=['product_id', 'unit_name'], right_on=['product_id', 'unit_name'],how='left')
# cleandata2['relative_quantity'] = cleandata2['quantity'].astype('float') / cleandata2['part_of_bigger_unit'].astype('float')
# cleandata2 = cleandata2.drop(columns = ['Unnamed: 0', 'subunit_per_unit'])


listofunits = cleandata[['product_id','unit_name']]
listofunits = listofunits.drop_duplicates(['product_id','unit_name'])
unitarray = listofunits['unit_name'].drop_duplicates().tolist()

import re

amountarray = []
for i in unitarray: 
    match = re.search(r'\d+',i)
    if match:
        amountarray.append(int(match.group()))
    else:
        if i == 'คู่': 
            amountarray.append(1)
        elif i == 'ลัง':
            amountarray.append(25) 

unitmatrix = dict(zip(unitarray,amountarray))
cleandata2 = cleandata

cleandata2['relative_unit'] = cleandata2['unit_name'].map(unitmatrix)
cleandata2['relative_quantity'] = cleandata2['quantity'].astype('float') * cleandata2['relative_unit']


After performing the necessary data manipulation, I have obtained the final dataset that we will use for creating the dashboard. The dataset now has accurate product IDs assigned to each entry, all cell values have been appropriately relocated to their respective columns

In [12]:
cleandata2

,invoice_date,invoiceid,customer_id,product_id,quantity,unit_name,unit_price,discount,revenue,relative_unit,relative_quantity
0,2019-01-12,DS256201/00119,C32,A153,20,คู่,32,-,640,1,20.0
1,2019-06-14,DS256206/00057,C2,A153,40,คู่,32,-,1280,1,40.0
2,2019-11-13,DS256211/00041,C32,A153,10,คู่,32,-,320,1,10.0
3,2019-11-14,DS256211/00043,C2,A153,1,ลัง40,1280,-,1280,40,40.0
4,2020-01-13,DS256301/00166,C21,A153,1,ลัง40,1280,-,1280,40,40.0
...,...,...,...,...,...,...,...,...,...,...,...
561,2021-12-22,DS256412/00056,C22,A163,1,ลัง,925,-,925,25,25.0
562,2021-12-24,DS256412/00085,C7,A163,3,ลัง,925,222,2553,25,75.0
563,2022-01-20,DS256501/00126,C35,A163,1,ลัง,925,-,925,25,25.0
564,2022-02-12,DS256502/00066,C68,A163,1,ลัง,925,-,925,25,25.0


Now that we have obtained a clean dataset, the next step is to create a sales dashboard using Power BI. The sales dashboard will provide an overview of sales performance by product, customer, and time, allowing the company to identify their top-selling products, high-value customers, and sales trends over time. Additionally, the dashboard will allow for easy updates, enabling the company to continuously monitor their sales performance and make data-driven decisions.

**Remark:** if you have already observe the actual sales dashboard, you may realized that the data that was used is not the same as the one that i use to demonstrate the python data manipulation. The data that was used in power bi demo is a dummy version with 2 years of randomized sales figure for privacy reason.

**Additional Notes:** In order to improve the ease of data cleaning for the client, I have gone the extra mile to create a (very rough) Graphical User Interface (GUI) for the data cleaning Python code. the sales manager will be able to easily and efficiently clean the data without needing any prior knowledge of Python and prepare it for input into a Power BI data folder.

the code for compile are presented below:

In [ ]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import csv 
from openpyxl import Workbook


root = tk.Tk()
root.title("CSV File Reader")
root.geometry("500x100")

    

def filecleaner():  
    if filepath:
        convert_label.config(text="Files Converted")
        
        with open(filepath, 'r', encoding='TIS-620') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            
            wb = Workbook()
            ws = wb.active
    
            for row in csv_reader:
                ws.append(row)
            
        sheet = wb[wb.sheetnames[0]]
        datasettest = pd.DataFrame(sheet.values)
        #datasettest = pd.DataFrame('', encoding='TIS-620')
            
        #datasettest = pd.read_excel(wb, sheet_name=0) #for excel file directory
        datasettest = datasettest.reset_index()
        datasettest = datasettest.drop(datasettest.columns[0:44], axis=1)
    
        cleandata = pd.DataFrame(columns=['Invoice_Date', 'InvoiceID', 'Customer_ID','Product_ID', 'Quantity','Unit', 'Unit_Price', 'Discount', 'Revenue'])  
    
        #iterator for data clense
        for i in range(0,datasettest.shape[0]):
            
            row = datasettest.iloc[i,1]
            
            if row.startswith(('A', 'B', 'C', 'T')): #if row has product category
                #inserting filtered data into new data
                productid = datasettest.iloc[i,1]
                cleandata.loc[i,'Invoice_Date'] = datasettest.iloc[i,3]
                cleandata.loc[i,'InvoiceID'] = datasettest.iloc[i,4]
                cleandata.loc[i,'Customer_ID'] = datasettest.iloc[i,5]
                cleandata.loc[i,'Product_ID'] = productid
                cleandata.loc[i,'Quantity'] = datasettest.iloc[i,8]
                cleandata.loc[i,'Unit'] = datasettest.iloc[i,7]
                cleandata.loc[i,'Unit_Price'] = datasettest.iloc[i,10]
                cleandata.loc[i,'Discount'] = datasettest.iloc[i,13]
                cleandata.loc[i,'Revenue'] = datasettest.iloc[i,14]
                
            
            elif row.startswith(("DS","VAT")): #if row has invoiceID
                cleandata.loc[i,'Invoice_Date'] = datasettest.iloc[i,0]
                cleandata.loc[i,'InvoiceID'] = datasettest.iloc[i,1]
                cleandata.loc[i,'Customer_ID'] = datasettest.iloc[i,2]
                cleandata.loc[i,'Product_ID'] = productid
                cleandata.loc[i,'Unit'] = datasettest.iloc[i,4]
                cleandata.loc[i,'Quantity'] = datasettest.iloc[i,5]
                cleandata.loc[i,'Unit_Price'] = datasettest.iloc[i,7]
                cleandata.loc[i,'Discount'] = datasettest.iloc[i,10]
                cleandata.loc[i,'Revenue'] = datasettest.iloc[i,11]
                
                
        #Get A Distinct Year Value from InvoiceDate
        distinctyear = cleandata['Invoice_Date'].str.slice(-4)
        distinctyear = distinctyear.drop_duplicates()
        distinctyear = distinctyear.tolist()
    
        #Loop for Conversion
        thai = sorted(distinctyear)
        chrs = []
    
        for i in thai:
            c = int(i) - 543
            chrs.append(str(c))
        year = dict(zip(thai,chrs))
    
    
        #Replacing Strings
        cleandata['Invoice_Date'] = cleandata['Invoice_Date'].replace(year,regex=True)
    
        #Convert String Invoice_Date to Datetime64[ns]
        cleandata['Invoice_Date'] = pd.to_datetime(cleandata['Invoice_Date'], format = '%d/%m/%Y', dayfirst=True)
        cleandata['Invoice_Date'] = cleandata['Invoice_Date'].dt.strftime('%d/%m/%Y')
    
        #fixing the unit issues
        
    
    
    
    
        #Export Data
        cleandata.to_excel('WP_cleaneddata.xlsx', index=False)
        
    else:
        input_path_label.config(text='Please Select Sales CSV Generated From BPLUS')



 
#function to get the file path 
def select_input_file():
    global filepath
    filepath = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    input_path_label.config(text=filepath) #return file path in button
    return filepath


        
    
#inserting a button
input_path_label  = tk.Button(root, text='Select file', command= lambda:select_input_file())
input_path_label.pack(padx=10, pady=10)

convert_label  = tk.Button(root, text='Convert', command=filecleaner)
convert_label.pack(padx=10, pady=10)


root.mainloop()